In [1]:
# Train a model.
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
clr = RandomForestClassifier()
clr.fit(X_train, y_train)

In [5]:

# Convert into ONNX format
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
initial_type = [('float_input', FloatTensorType([None, 4]))]
onx = convert_sklearn(clr, initial_types=initial_type)
with open("rf_iris.onnx", "wb") as f:
    f.write(onx.SerializeToString())


In [2]:
# Compute the prediction with ONNX Runtime
import onnxruntime as rt
import numpy
sess = rt.InferenceSession("rf_iris.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onnx = sess.run([label_name], {input_name: X_test.astype(numpy.float32)})[0]

In [3]:
pred_onnx

array([1, 0, 0, 2, 1, 0, 0, 1, 2, 2, 2, 1, 0, 1, 1, 2, 1, 0, 2, 0, 2, 0,
       0, 0, 1, 1, 0, 1, 1, 2, 2, 2, 1, 1, 2, 1, 0, 1], dtype=int64)

In [8]:
import pandas as pd
df =pd.read_csv('input_data.csv')

In [9]:
df.head()

,x,x2,x3,x4
0,4.9,2.4,3.3,1.0
1,4.6,3.6,1.0,0.2
2,5.4,3.4,1.5,0.4
3,5.8,2.7,5.1,1.9
4,6.3,3.3,4.7,1.6


In [15]:
dfarray = numpy.array(df)

In [37]:
pred_onnx = sess.run([label_name], {input_name: dfarray.astype(numpy.float32)})[0]
outdf = pd.DataFrame(columns=['id','prediction'])
outdf['id']=range(len(dfarray))
outdf['prediction'] = pred_onnx
outdf.to_dict(orient='records')

In [38]:
#.to_dict(orient='records')

In [39]:
pred_onnx

array([1, 0, 0, 2, 1, 0, 0, 1, 2, 2, 2, 1, 0, 1, 1, 2, 1, 0, 2, 0, 2, 0,
       0, 0, 1, 1, 0, 1, 1, 2, 2, 2, 1, 1, 2, 1, 0, 1], dtype=int64)

[{'id': 0, 'prediction': 1},
 {'id': 1, 'prediction': 0},
 {'id': 2, 'prediction': 0},
 {'id': 3, 'prediction': 2},
 {'id': 4, 'prediction': 1},
 {'id': 5, 'prediction': 0},
 {'id': 6, 'prediction': 0},
 {'id': 7, 'prediction': 1},
 {'id': 8, 'prediction': 2},
 {'id': 9, 'prediction': 2},
 {'id': 10, 'prediction': 2},
 {'id': 11, 'prediction': 1},
 {'id': 12, 'prediction': 0},
 {'id': 13, 'prediction': 1},
 {'id': 14, 'prediction': 1},
 {'id': 15, 'prediction': 2},
 {'id': 16, 'prediction': 1},
 {'id': 17, 'prediction': 0},
 {'id': 18, 'prediction': 2},
 {'id': 19, 'prediction': 0},
 {'id': 20, 'prediction': 2},
 {'id': 21, 'prediction': 0},
 {'id': 22, 'prediction': 0},
 {'id': 23, 'prediction': 0},
 {'id': 24, 'prediction': 1},
 {'id': 25, 'prediction': 1},
 {'id': 26, 'prediction': 0},
 {'id': 27, 'prediction': 1},
 {'id': 28, 'prediction': 1},
 {'id': 29, 'prediction': 2},
 {'id': 30, 'prediction': 2},
 {'id': 31, 'prediction': 2},
 {'id': 32, 'prediction': 1},
 {'id': 33, 'predict